In [4]:
import sys
sys.path.append("../Scripts/")

from pdf_parser import sentence_parser 

In [5]:
pdf = '../SampleReports/2023_Coles_Report.pdf'

In [6]:
sentence_parser(pdf)

Report located in specified directory

Summary:
Total number of sentences: 686


['2023 Sustainability ReportWorking towards a more sustainable future Coles Group Limited ABN 11 004 089 936Acknowledgement of Country Coles wishes to acknowledge the Traditional Custodians of Country throughout Australia.',
 'We recognise their strength and resilience and pay our respects to their Elders past and present.',
 'Coles extends that respect to all Aboriginal and Torres Strait Islander people, and recognises their rich cultures and their continuing connection to land and waters.',
 'Aboriginal and Torres Strait Islander people are advised that this report may contain names and images of people who are deceased.',
 'All references to Indigenous and First Nations people in this report are intended to include Aboriginal and/or Torres Strait Islander people.',
 'Feedback We welcome feedback on this report.',
 'This report also includes forward-looking statements regarding climate change and other environmental and energy transition scenarios.',
 'Forward-looking statements can 